<a href="https://colab.research.google.com/github/Vilmo18/Fine-Tuning-LLMs/blob/main/NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install flair
!pip install datasets
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.3/388.3 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 47.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 46.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.6/202.6 kB 15.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.9/143.9 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 47.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 9.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-n

In [2]:
from flair.data import Sentence
from flair.models import SequenceTagger

In [3]:
tagger = SequenceTagger.load('flair/ner-english-ontonotes-large')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

2024-07-01 16:43:46,319 SequenceTagger predicts: Dictionary with 76 tags: <unk>, O, B-CARDINAL, E-CARDINAL, S-PERSON, S-CARDINAL, S-PRODUCT, B-PRODUCT, I-PRODUCT, E-PRODUCT, B-WORK_OF_ART, I-WORK_OF_ART, E-WORK_OF_ART, B-PERSON, E-PERSON, S-GPE, B-DATE, I-DATE, E-DATE, S-ORDINAL, S-LANGUAGE, I-PERSON, S-EVENT, S-DATE, B-QUANTITY, E-QUANTITY, S-TIME, B-TIME, I-TIME, E-TIME, B-GPE, E-GPE, S-ORG, I-GPE, S-NORP, B-FAC, I-FAC, E-FAC, B-NORP, E-NORP, S-PERCENT, B-ORG, E-ORG, B-LANGUAGE, E-LANGUAGE, I-CARDINAL, I-ORG, S-WORK_OF_ART, I-QUANTITY, B-MONEY


In [4]:
included_tags = {
    "EVENT", "FAC", "GPE", "LANGUAGE", "LAW", "LOCATION", "NORP",
    "ORG", "PERSON", "PRODUCT", "WORK_OF_ART"
}

excluded_tags = {
    "CARDINAL", "DATE", "MONEY", "ORDINAL", "PERCENT", "QUANTITY", "TIME"
}

def anonymize_entities(text):
    sentence = Sentence(text)
    tagger.predict(sentence)

    token_replacements = {}

    for entity in sentence.get_spans('ner'):
        if entity.tag in included_tags:
            #Replace the entity with its tag
            token_replacements[(entity.start_position, entity.end_position)] = f"[{entity.tag}]"

    #Create anonymized text by replacing entities with their tags
    anonymized_text = ''
    last_idx = 0

    for (start, end), replacement in sorted(token_replacements.items()):
        anonymized_text += text[last_idx:start] + replacement
        last_idx = end

    anonymized_text += text[last_idx:]

    return anonymized_text


text = "Francesco Totti threatens to quit the Italian Players' Association . The Italy and Roma striker is upset over the early start to the new season . He says: We are the principal actors, but ... are never listened to."
anonymized_text = anonymize_entities(text)
print(anonymized_text)


[PERSON] threatens to quit [ORG] . The [GPE] and [GPE] striker is upset over the early start to the new season . He says: We are the principal actors, but ... are never listened to.


In [5]:
from datasets import load_dataset
ds = load_dataset("abisee/cnn_dailymail", "3.0.0")

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [6]:
ds

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

In [7]:
with open("article.txt", "w") as articles , open("highlights.txt", "w") as highlights:
    for it in ds["train"]:
        articles.write(it["article"] + "\n")
        highlights.write(it["highlights"] + "\n")

In [8]:
!wget https://raw.githubusercontent.com/google/sentencepiece/master/data/botchan.txt

--2024-07-01 16:44:36--  https://raw.githubusercontent.com/google/sentencepiece/master/data/botchan.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 278779 (272K) [text/plain]
Saving to: ‘botchan.txt’

botchan.txt         100%[===================>] 272.25K  --.-KB/s    in 0.02s   

2024-07-01 16:44:37 (11.6 MB/s) - ‘botchan.txt’ saved [278779/278779]



In [12]:
import sentencepiece as spm

# train sentencepiece model from `botchan.txt` and makes `m.model` and `m.vocab`
# `m.vocab` is just a reference. not used in the segmentation.
# spm.SentencePieceTrainer.train('--input=botchan.txt')

# "EVENT", "FAC", "GPE", "LANGUAGE", "LAW", "LOCATION", "NORP",
    # "ORG", "PERSON", "PRODUCT", "WORK_OF_ART"

spm.SentencePieceTrainer.Train('--input=highlights.txt --model_prefix=m   --user_defined_symbols=[PERSON],[ORG],[GPE]')
# makes segmenter instance and loads the model file (m.model)
sp = spm.SentencePieceProcessor()
sp.load('m.model')

# encode: text => id
#text='[PERSON] threatens to quit [ORG] '
text='[PERSON] threatens to quit [ORG]. The [GPE] and [GPE] striker is upset over the early start to the new season . He says: We are the principal actors, but ... are never listened to.'
print(sp.encode_as_pieces(text))
print(sp.encode_as_ids(text))

# decode: id => text
#print(sp.decode_pieces(['▁This', '▁is', '▁a', '▁t', 'est']))
print(sp.decode_ids(sp.encode_as_ids(text)))

['▁', '[PERSON]', '▁threat', 'ens', '▁to', '▁quit', '▁', '[ORG]', '.', '▁The', '▁', '[GPE]', '▁and', '▁', '[GPE]', '▁striker', '▁is', '▁upset', '▁over', '▁the', '▁early', '▁start', '▁to', '▁the', '▁new', '▁season', '▁.', '▁He', '▁says', ':', '▁We', '▁are', '▁the', '▁principal', '▁actor', 's', ',', '▁but', '▁...', '▁are', '▁never', '▁listen', 'ed', '▁to', '.']
[16, 3, 1406, 1341, 10, 2508, 16, 4, 32, 24, 16, 5, 15, 16, 5, 948, 23, 4084, 77, 8, 704, 625, 10, 8, 104, 325, 6, 52, 36, 49, 671, 44, 8, 6686, 1900, 7, 9, 67, 3396, 44, 618, 4851, 20, 10, 32]
[PERSON] threatens to quit [ORG]. The [GPE] and [GPE] striker is upset over the early start to the new season . He says: We are the principal actors, but ... are never listened to.


In [10]:
# encode: text => id
print(sp.encode_as_pieces(anonymized_text))

['▁', '[PERSON]', '▁threatens', '▁to', '▁quit', '▁', '[ORG]', '▁.', '▁The', '▁', '[GPE]', '▁and', '▁', '[GPE]', '▁striker', '▁is', '▁upset', '▁over', '▁the', '▁early', '▁start', '▁to', '▁the', '▁new', '▁season', '▁.', '▁He', '▁says', ':', '▁We', '▁are', '▁the', '▁principal', '▁actors', ',', '▁but', '▁...', '▁are', '▁never', '▁listened', '▁to', '.']


In [11]:
anonymized_text

'[PERSON] threatens to quit [ORG] . The [GPE] and [GPE] striker is upset over the early start to the new season . He says: We are the principal actors, but ... are never listened to.'